# Imports

In [121]:
import  tensorflow                  as      tf
import  numpy                       as      np
import  os
from    sklearn.model_selection     import  train_test_split
from    tensorflow.keras.utils      import  to_categorical

from    tensorflow.keras.models     import  Sequential
from    tensorflow.keras.layers     import  LSTM, Dense
from    tensorflow.keras.callbacks  import  TensorBoard

## Constants

In [122]:
SAMPLES_PER_WORD = 60
FRAMES_PER_SAMPLE = 30

SAMPLES_PATH = "/Volumes/SSD/collection/"

In [124]:

words = np.array(['none', 'person', 'hello', 'school', 'i love you'])
words_path = {
    'hello' : 59,
    'none' : 61,
    'person' : 63,
    'school' : 67,
    'i love you' : 69 
    }

labels_map = {label:num for num, label in enumerate(words)}
labels_map

{'none': 0, 'person': 1, 'hello': 2, 'school': 3, 'i love you': 4}

In [126]:

samples, labels = [], []
for word in words:
    for sample in range(SAMPLES_PER_WORD):
        sample_data = []
        for frame in range(FRAMES_PER_SAMPLE):
            path        = os.path.join(SAMPLES_PATH, f'{words_path[word]}', f'{words_path[word]}_{sample}_{frame}.npy')
            frame_data  = np.load(path)
            sample_data.append(frame_data)
        samples.append(sample_data)
        labels.append(labels_map[word])

In [127]:
X = np.array(samples)
X.shape

(300, 30, 225)

In [128]:
y = to_categorical(labels).astype(int)

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Model

In [130]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model           = Sequential()

hands   = 63 + 63
pose    = 99

lstm_model_1    = LSTM(     64,     return_sequences = True,    activation = "relu", input_shape=(30, hands+pose) )
lstm_model_2    = LSTM(     128,    return_sequences = True,    activation = "relu" )
lstm_model_3    = LSTM(     64,     return_sequences = False,   activation = "relu" )

dense_model_1   = Dense(    64,                                 activation = "relu" )
dense_model_2   = Dense(    32,                                 activation = "relu" )
dense_model_3   = Dense(    words.shape[0],                     activation = "softmax" )

model.add( lstm_model_1 )
model.add( lstm_model_2 )
model.add( lstm_model_3 )

model.add( dense_model_1 )
model.add( dense_model_2 )
model.add( dense_model_3 )

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [131]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [132]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - categorical_accuracy: 0.2329 - loss: 1.5418
Epoch 2/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.4024 - loss: 1.2524
Epoch 3/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.4085 - loss: 1.3421
Epoch 4/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.6109 - loss: 1.0469
Epoch 5/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.6362 - loss: 0.8436
Epoch 6/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.6072 - loss: 1.0455
Epoch 7/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.6780 - loss: 0.8883
Epoch 8/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.6175 - loss: 0.8481
Epoch 9/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.7782 - loss: 0.6096
Epoch 10/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - categorical_accuracy: 0.6990 - loss: 0.7064
Epoch 11/1000
9/9 ━━━━━━━━━━━

In [120]:
model.save('new2.keras')